In [1]:
%cd ~/content/jepa

!python setup.py install


/home/sagemaker-user/content/jepa
running install
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standar

In [4]:
%cd ~/content/jepa

import sagemaker
from sagemaker.pytorch import PyTorch, TrainingCompilerConfig
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput

role = sagemaker.get_execution_role()
hyperparameters={
    "fname": "configs/evals/vitl16_in1k.yaml",
    "devices": ['cuda:0'],
}

pytorch_estimator=PyTorch(
    entry_point='evals_main.py',
    role=role,
    source_dir='/home/sagemaker-user/content/jepa', # Optional. Add this if need to install additional packages.
    instance_count=1,
    instance_type='ml.g4dn.xlarge', # ml.m5.xlarge, ml.g4dn.xlarge(現状はGPUのある ml.g4dn.xlarge でないと上手くいかない、https://github.com/facebookresearch/jepa/issues/55 の対応すれば ml.m5.xlarge でも上手く行く)
    framework_version='1.13.1',
    py_version='py39',
    hyperparameters=hyperparameters,
    disable_profiler=True,
    debugger_hook_config=False
)

# この引数を足すと以下エラーが発生する。500GBでやろうとしたが125GBまでしか大きくできない旨のエラーが出た。
# ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Invalid VolumeSizeInGB: 500 GB. The requested instance type ml.g4dn.xlarge includes local instance storage with a fixed total size of 125 GB. Please reduce VolumeSizeInGB in the request or choose a different instance type.
# 直そうと思ったが、作業を中断してしまったので直していない
#    volume_size=500,


# この引数を足すと以下エラーが発生する。jepaで分散トレーニングをしないように修正したのを戻したがやはり下記エラーが発生したので、そのまま下記引数は足さない
# [エラー内容]AttributeError: module 'torch' has no attribute '_six'
#    compiler_config=TrainingCompilerConfig(),

pytorch_estimator.fit(
    inputs={
        "pretrained_model": TrainingInput(s3_data="s3://jepa-vitl16"),
        "train": TrainingInput(s3_data="s3://jepa-images/train"),
        "val": TrainingInput(s3_data="s3://jepa-images/val"),
    }
)

/home/sagemaker-user/content/jepa


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-07-12-12-10-09-779


2024-07-12 12:13:41 Starting - Starting the training job...
2024-07-12 12:13:58 Starting - Preparing the instances for training...
2024-07-12 12:14:26 Downloading - Downloading input data.........
2024-07-12 12:15:57 Downloading - Downloading the training image...............
2024-07-12 12:18:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-12 12:18:33,312 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-12 12:18:33,333 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-07-12 12:18:33,344 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-12 12:18:33,348 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-07-12 12:19:03,650 sagemaker-training-toolkit INFO     Installing module with

In [ ]:
%cd ~/content/jepa
!python -m app.main --fname configs/pretrain/vitl16.yaml --devices cuda:0 cuda:1 cuda:2

In [25]:
%cd ~/content/jepa
!python -m evals_main --fname configs/evals/vitl16_in1k.yaml --devices cuda:0 #cuda:1 cuda:2

/home/sagemaker-user/content/jepa
INFO:root:called-params configs/evals/vitl16_in1k.yaml
INFO:root:loaded params...
{   'data': {   'dataset_name': 'ImageNet',
                'image_folder': 'imagenet_full_size/061417/',
                'num_classes': 1,
                'resolution': 224,
                'root_path': '/MyDrive/jepa_images/'},
    'eval_name': 'image_classification_frozen',
    'nodes': 8,
    'optimization': {   'batch_size': 16,
                        'final_lr': 0.0,
                        'lr': 0.001,
                        'num_epochs': 20,
                        'start_lr': 0.001,
                        'use_bfloat16': True,
                        'warmup': 0.0,
                        'weight_decay': 0.001},
    'pretrain': {   'checkpoint': 'vitl16.pth.tar',
                    'checkpoint_key': 'target_encoder',
                    'clip_duration': None,
                    'folder': '/MyDrive/',
                    'frames_per_clip': 16,
               

In [ ]:
import torch

# モデルの読み込み
PATH = '/content/drive/MyDrive/vitl16.pth.tar'  # モデルファイルへのパスを指定
my_dict = torch.load(PATH, map_location=torch.device('cpu'))

classifier=my_dict['target_encoder']

print(f"Dictionary contents: {classifier}")

Dictionary contents: OrderedDict([('module.backbone.pos_embed', tensor([[[0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.6570, 0.3427, 0.0019,  ..., 0.9999, 0.9999, 1.0000],
         [0.6570, 0.3427, 0.0019,  ..., 0.9999, 0.9999, 0.9999],
         [0.6570, 0.3427, 0.0019,  ..., 0.9999, 0.9999, 0.9999]]])), ('module.backbone.patch_embed.proj.weight', tensor([[[[[ 1.6566e-03, -2.3490e-03, -1.1865e-03,  ..., -8.6622e-04,
            -2.3996e-04, -1.1941e-02],
           [ 5.3274e-03,  5.2559e-04,  8.7542e-04,  ...,  6.2355e-04,
            -1.8926e-04, -9.8496e-03],
           [ 3.3134e-03, -5.0810e-04, -1.3705e-04,  ..., -2.2038e-03,
            -1.4955e-03, -1.1411e-02],
           ...,
           [ 9.2960e-03,  1.0699e-03,  6.2105e-04,  ...,  2.1257e-04,
             1.0443e-03, -6.9656e-03],
           [ 7.1181e-03, -1.4097

In [1]:
%cd ~/
!df -h

/home/sagemaker-user/content/drive/MyDrive/jepa_images
Filesystem      Size  Used Avail Use% Mounted on
overlay          32G   52K   32G   1% /
tmpfs            64M     0   64M   0% /dev
tmpfs           1.9G     0  1.9G   0% /sys/fs/cgroup
shm             392M   20K  392M   1% /dev/shm
/dev/nvme0n1p1  160G   32G  129G  20% /opt/.sagemakerinternal
/dev/nvme1n1    500G  8.6G  492G   2% /home/sagemaker-user
tmpfs           1.9G     0  1.9G   0% /proc/acpi
tmpfs           1.9G     0  1.9G   0% /sys/firmware
